In [48]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False
import re
import csv
import urllib
import requests
import time
from collections import defaultdict

def get_weibo_html(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        return None
    except ConnectionError:
        print("Error occurred")
        return None

def get_words_nums(html):
#      热搜词的正则表达式
    hot_key_words = re.compile(r'td class=\\"td_05\\"><a href=\\"\\/weibo\\/(.*?)&Refer=top\\"')
#      热搜词对应搜多热度的正则表达式
    hot_key_nums = re.compile(r'td class=\\"td_03\\"><p class=\\"star_num\\"><span>(.*?)<\\/span><\\/p>')
#      分别获得热搜词和搜索热度的元数据
    data_words = hot_key_words.findall(html)
    data_nums = hot_key_nums.findall(html)
#      分别获得热搜词和搜索热度的最终数据
    data_words_final = []
    for i in data_words:
        data_words_final.append(urllib.parse.unquote(urllib.parse.unquote(i)).replace('+',' '))
    data_nums_final = []
    for i in data_nums:
        data_nums_final.append(int(i))
        
    return data_words_final,data_nums_final

def plot_barh(words,nums):
    plt.xlabel(u'搜索热度')
    plt.ylabel(u'热搜词')
    plt.barh(range(10),nums[:10],color='rgb',tick_label=words[:10])

#     收集热搜数据
def collect_words_nums(words,nums,dicts):
    for key,num in zip(words,nums):
        dicts[key].append(num)
        
#     保存在本地，两个文件，这一个记录所有的搜索热度
def save_files_all(dicts):
    with open('all_csv','a') as csv_file:
        writer = csv.writer(csv_file)
        for key , value in dicts.items():
            if type(value)==int:
                writer.writerow([key,value])
            else:
                writer.writerow([key,*value])

#     保存在本地，这一个记录最高的搜索热度   
def save_files_most(dicts):
    with open('most_csv','a') as csv_file:
        writer = csv.writer(csv_file)
        for key , value in dicts.items():
            if type(value)==int:
                writer.writerow([key,value])
            else:
                writer.writerow([key,max(value)])
    
def main():
    url = 'http://s.weibo.com/top/summary?cate=realtimehot'
    dicts = defaultdict(list)
    localtime = time.asctime(time.localtime(time.time()))
    n = 0
    while(n <= 1):
        html = get_weibo_html(url)
        words,data = get_words_nums(html)
        collect_words_nums(words,data,dicts)
        save_files_all(dicts)
        plt.show()
        n = n + 1
        time.sleep(60)
    save_files_most(dicts) 

    
# 运行！
main()

In [49]:
#  直接点击上面的  Run 即可运行。